# Node Embeddings Example: Node2Vec Graph Structure-Aware Embeddings

This notebook demonstrates how to:
1. Build a knowledge graph with nodes and edges
2. Compute Node2Vec embeddings that capture graph structure
3. Query nodes by structural similarity using embeddings
4. Understand how Node2Vec embeddings differ from text-based embeddings
5. Use embeddings for similarity search and clustering

Node2Vec embeddings are structure-aware, meaning nodes with similar connection patterns will have similar embeddings, regardless of their text attributes.


In [1]:
import os
import json
import numpy as np
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from spindle import (
    SpindleExtractor,
    create_ontology,
    GraphStore,
    ChromaVectorStore
)

# Load environment variables
load_dotenv()

# Check if required packages are available
try:
    import networkx as nx
    from node2vec import Node2Vec
    print("✓ Required packages (networkx, node2vec) are available")
except ImportError as e:
    print(f"✗ Missing required package: {e}")
    print("Install with: pip install node2vec networkx")


✓ Required packages (networkx, node2vec) are available


## Part 1: Building a Knowledge Graph

First, we need to build a knowledge graph with nodes and edges. Node2Vec embeddings are computed from the graph structure, so we need a graph with connections before we can compute embeddings.


In [2]:
# Create a GraphStore and build a sample knowledge graph
graph_name = f"embeddings_example_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"Creating graph: {graph_name}")

with GraphStore(db_path=graph_name) as store:
    # Add nodes representing people and organizations
    store.add_node("Alice Johnson", "Person", description="Senior software engineer")
    store.add_node("Bob Smith", "Person", description="Data scientist")
    store.add_node("Carol Davis", "Person", description="Product manager")
    store.add_node("David Chen", "Person", description="DevOps engineer")
    
    store.add_node("TechCorp", "Organization", description="Technology company")
    store.add_node("DataSystems Inc", "Organization", description="Data analytics company")
    store.add_node("CloudTech", "Organization", description="Cloud services provider")
    
    # Add edges to create connections
    store.add_edge("Alice Johnson", "works_at", "TechCorp")
    store.add_edge("Bob Smith", "works_at", "TechCorp")
    store.add_edge("Carol Davis", "works_at", "TechCorp")
    store.add_edge("David Chen", "works_at", "CloudTech")
    
    store.add_edge("Bob Smith", "works_at", "DataSystems Inc")
    
    # Add some relationships between people
    store.add_edge("Alice Johnson", "knows", "Bob Smith")
    store.add_edge("Alice Johnson", "knows", "Carol Davis")
    store.add_edge("Bob Smith", "knows", "Carol Davis")
    
    print("✓ Graph created with nodes and edges")
    
    # Get statistics
    stats = store.get_statistics()
    print(f"\nGraph Statistics:")
    print(f"  Nodes: {stats['node_count']}")
    print(f"  Edges: {stats['edge_count']}")


Creating graph: embeddings_example_20251105_164041
✓ Graph created with nodes and edges

Graph Statistics:
  Nodes: 7
  Edges: 8


## Part 2: Computing Node2Vec Embeddings

Now that we have a graph, we can compute Node2Vec embeddings. These embeddings capture the structural relationships between nodes - nodes with similar connection patterns will have similar embeddings.


In [4]:
# Create VectorStore for storing embeddings
vector_store = ChromaVectorStore(collection_name="node2vec_embeddings")

# Compute Node2Vec embeddings
print("Computing Node2Vec embeddings...")
with GraphStore(db_path=graph_name) as store:
    embeddings = store.compute_graph_embeddings(
        vector_store,
        dimensions=128,      # Embedding dimension
        walk_length=80,      # Length of random walks
        num_walks=10,        # Number of walks per node
        p=1.0,               # Return parameter
        q=1.0                # In-out parameter
    )
    
    print(f"✓ Computed embeddings for {len(embeddings)} nodes")
    print(f"  Embedding dimension: 128")
    print(f"\nNode -> Vector Index mapping:")
    for node_name, vector_index in list(embeddings.items())[:5]:  # Show first 5
        print(f"  {node_name}: {vector_index[:20]}...")


Computing Node2Vec embeddings...


Computing transition probabilities:   0%|          | 0/7 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 573.26it/s]


✓ Computed embeddings for 7 nodes
  Embedding dimension: 128

Node -> Vector Index mapping:
  ALICE JOHNSON: 63635159-5710-42e1-8...
  BOB SMITH: d56322ed-5276-4072-9...
  CAROL DAVIS: d0eda80d-2aaa-4b4e-9...
  DAVID CHEN: abda30ab-76e0-4116-a...
  TECHCORP: 9121f6ad-3311-417e-b...


## Part 3: Inspecting Node Embeddings

Let's check that nodes now have embeddings and see what they look like.


In [10]:
# Helper function to query by embedding vector (for Node2Vec embeddings)
from typing import List, Dict, Any

def query_by_node_embedding(
    vector_store: ChromaVectorStore,
    query_node_name: str,
    graph_store: GraphStore,
    top_k: int = 5,
    exclude_query_node: bool = True
) -> List[Dict[str, Any]]:
    """
    Find nodes structurally similar to a given node using Node2Vec embeddings.
    
    Args:
        vector_store: VectorStore instance
        query_node_name: Name of the node to find similar nodes for
        graph_store: GraphStore instance
        top_k: Number of similar nodes to return
        exclude_query_node: Whether to exclude the query node from results
    
    Returns:
        List of similar nodes with their similarity scores
    """
    # Get the query node's embedding
    query_node = graph_store.get_node(query_node_name)
    if not query_node or not query_node.get('vector_index'):
        return []
    
    query_embedding_data = vector_store.get(query_node['vector_index'])
    if not query_embedding_data:
        return []
    
    query_embedding = query_embedding_data['embedding']
    
    # Query ChromaDB collection directly by embedding vector
    # Note: ids are always returned, so we don't need to include them explicitly
    results = vector_store.collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k + 1,  # Get one extra in case we need to exclude query node
        where={"type": "node"},
        include=["documents", "metadatas", "distances"]
    )
    
    # Format results
    similar_nodes = []
    for i in range(len(results["ids"][0])):
        node_name = results["metadatas"][0][i].get("name", "")
        
        # Skip query node if requested
        if exclude_query_node and node_name.upper() == query_node_name.upper():
            continue
        
        similar_nodes.append({
            "name": node_name,
            "type": results["metadatas"][0][i].get("entity_type", ""),
            "distance": results["distances"][0][i],
            "vector_index": results["ids"][0][i]
        })
        
        if len(similar_nodes) >= top_k:
            break
    
    return similar_nodes

# Check that nodes have embeddings
with GraphStore(db_path=graph_name) as store:
    alice = store.get_node("Alice Johnson")
    
    if alice and alice.get("vector_index"):
        print(f"Node: {alice['name']}")
        print(f"Type: {alice['type']}")
        print(f"Vector Index: {alice['vector_index'][:30]}...")
        print()
        
        # Retrieve the embedding from vector store
        embedding_data = vector_store.get(alice['vector_index'])
        if embedding_data:
            print("Embedding Information:")
            print(f"  Dimension: {len(embedding_data['embedding'])}")
            print(f"  Metadata: {json.dumps(embedding_data['metadata'], indent=2)}")
            print(f"  First 10 values: {[round(x, 4) for x in embedding_data['embedding'][:10]]}")
    else:
        print("Node not found or no embedding")


Node: ALICE JOHNSON
Type: Person
Vector Index: 63635159-5710-42e1-854c-bc6656...

Embedding Information:
  Dimension: 128
  Metadata: {
  "entity_type": "Person",
  "dimensions": 128,
  "name": "ALICE JOHNSON",
  "type": "node",
  "embedding_method": "node2vec"
}
  First 10 values: [-0.0343, -0.1868, 0.2308, -0.0416, -0.0476, -0.2442, 0.0496, 0.0472, -0.1856, 0.0683]


## Part 4: Querying Nodes by Structural Similarity

Node2Vec embeddings capture structural relationships in the graph. Nodes with similar connection patterns will have similar embeddings. We can find structurally similar nodes by querying using one node's embedding as the query vector.

**Important**: Unlike text-based embeddings, Node2Vec embeddings are structure-aware, so we query by using an existing node's embedding rather than text.


In [11]:
# Query 1: Find nodes structurally similar to "Alice Johnson"
print("Query 1: Finding nodes structurally similar to 'Alice Johnson'")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    similar = query_by_node_embedding(vector_store, "Alice Johnson", store, top_k=5)
    
    print(f"Found {len(similar)} structurally similar node(s):\n")
    for i, result in enumerate(similar, 1):
        node = store.get_node(result['name'])
        print(f"{i}. {result['name']} ({result['type']})")
        if node:
            print(f"   Description: {node.get('description', 'N/A')}")
        print(f"   Structural similarity distance: {result['distance']:.4f} (lower = more similar)")
        print()


Query 1: Finding nodes structurally similar to 'Alice Johnson'
----------------------------------------------------------------------
Found 5 structurally similar node(s):

1. DATASYSTEMS INC (Organization)
   Description: Data analytics company
   Structural similarity distance: 0.0007 (lower = more similar)

2. BOB SMITH (Person)
   Description: Data scientist
   Structural similarity distance: 0.0012 (lower = more similar)

3. TECHCORP (Organization)
   Description: Technology company
   Structural similarity distance: 0.0014 (lower = more similar)

4. CAROL DAVIS (Person)
   Description: Product manager
   Structural similarity distance: 0.0016 (lower = more similar)

5. DAVID CHEN (Person)
   Description: DevOps engineer
   Structural similarity distance: 0.8768 (lower = more similar)



In [13]:
# Query 2: Find nodes structurally similar to "TechCorp" (an organization)
print("Query 2: Finding nodes structurally similar to 'TechCorp'")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    similar = query_by_node_embedding(vector_store, "TechCorp", store, top_k=3)
    
    print(f"Found {len(similar)} structurally similar node(s):\n")
    for i, result in enumerate(similar, 1):
        node = store.get_node(result['name'])
        print(f"{i}. {result['name']} ({result['type']})")
        if node:
            print(f"   Description: {node.get('description', 'N/A')}")
        print(f"   Structural similarity distance: {result['distance']:.4f}")
        print()
    
    # Show why they're similar - check their connections
    print("Why these nodes are similar (structural patterns):")
    techcorp = store.get_node("TechCorp")
    if techcorp:
        # Get connections for TechCorp
        connections = store.query_cypher("""
            MATCH (t:Entity {name: 'TECHCORP'})-[r:Relationship]-(other:Entity)
            RETURN type(r) as rel_type, other.name as connected_to
        """)
        print(f"  TechCorp is connected to: {[c['connected_to'] for c in connections]}")
        print()


Query 2: Finding nodes structurally similar to 'TechCorp'
----------------------------------------------------------------------
Found 3 structurally similar node(s):

1. BOB SMITH (Person)
   Description: Data scientist
   Structural similarity distance: 0.0006

2. CAROL DAVIS (Person)
   Description: Product manager
   Structural similarity distance: 0.0008

3. DATASYSTEMS INC (Organization)
   Description: Data analytics company
   Structural similarity distance: 0.0012

Why these nodes are similar (structural patterns):
  TechCorp is connected to: []



In [15]:
# Query 3: Find Person nodes structurally similar to "Bob Smith"
print("Query 3: Finding Person nodes structurally similar to 'Bob Smith'")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    similar = query_by_node_embedding(vector_store, "Bob Smith", store, top_k=5)
    
    # Filter to only Person nodes
    person_similar = [s for s in similar if s['type'] == 'Person']
    
    print(f"Found {len(person_similar)} Person node(s) structurally similar to Bob Smith:\n")
    for i, result in enumerate(person_similar, 1):
        node = store.get_node(result['name'])
        print(f"{i}. {result['name']} ({result['type']})")
        if node:
            print(f"   Description: {node.get('description', 'N/A')}")
        print(f"   Structural similarity distance: {result['distance']:.4f}")
        
        # Show connections to understand structural similarity
        node_name_upper = result['name'].upper()
        connections = store.query_cypher(f"""
            MATCH (n:Entity {{name: '{node_name_upper}'}})-[r:Relationship]-(other:Entity)
            RETURN type(r) as rel_type, other.name as connected_to
        """)
        if connections:
            conn_strs = [f"{c['rel_type']} -> {c['connected_to']}" for c in connections[:3]]
            print(f"   Connections: {conn_strs}")
        print()


Query 3: Finding Person nodes structurally similar to 'Bob Smith'
----------------------------------------------------------------------
Found 3 Person node(s) structurally similar to Bob Smith:

1. CAROL DAVIS (Person)
   Description: Product manager
   Structural similarity distance: 0.0008

2. ALICE JOHNSON (Person)
   Description: Senior software engineer
   Structural similarity distance: 0.0012

3. DAVID CHEN (Person)
   Description: DevOps engineer
   Structural similarity distance: 0.9019



## Part 5: Understanding Structural Similarity

Node2Vec embeddings capture structural roles in the graph. Nodes with similar connection patterns (same number of neighbors, similar roles) will have similar embeddings, even if they represent different entities. Let's explore what makes nodes structurally similar.


In [16]:
# Analyze structural patterns for nodes
print("Analyzing structural patterns to understand Node2Vec similarity:")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    # Compare nodes that are structurally similar
    alice = store.get_node("Alice Johnson")
    bob = store.get_node("Bob Smith")
    
    if alice and bob:
        # Get their connections
        alice_connections = store.query_cypher("""
            MATCH (a:Entity {name: 'ALICE JOHNSON'})-[r:Relationship]-(other:Entity)
            RETURN type(r) as rel_type, other.name as connected_to, labels(other)[0] as other_type
            ORDER BY other.name
        """)
        
        bob_connections = store.query_cypher("""
            MATCH (b:Entity {name: 'BOB SMITH'})-[r:Relationship]-(other:Entity)
            RETURN type(r) as rel_type, other.name as connected_to, labels(other)[0] as other_type
            ORDER BY other.name
        """)
        
        print(f"Alice Johnson's connections:")
        for conn in alice_connections:
            print(f"  - {conn['rel_type']} -> {conn['connected_to']} ({conn['other_type']})")
        
        print(f"\nBob Smith's connections:")
        for conn in bob_connections:
            print(f"  - {conn['rel_type']} -> {conn['connected_to']} ({conn['other_type']})")
        
        # Find their structural similarity
        similar_to_alice = query_by_node_embedding(vector_store, "Alice Johnson", store, top_k=3)
        print(f"\nNodes structurally similar to Alice Johnson:")
        for result in similar_to_alice:
            print(f"  - {result['name']} (distance: {result['distance']:.4f})")
        
        print("\nNote: Nodes with similar connection patterns (e.g., both connected to TechCorp,")
        print("both have 'knows' relationships) will have similar Node2Vec embeddings.")


Analyzing structural patterns to understand Node2Vec similarity:
----------------------------------------------------------------------
Alice Johnson's connections:

Bob Smith's connections:

Nodes structurally similar to Alice Johnson:
  - DATASYSTEMS INC (distance: 0.0007)
  - BOB SMITH (distance: 0.0012)
  - TECHCORP (distance: 0.0014)

Note: Nodes with similar connection patterns (e.g., both connected to TechCorp,
both have 'knows' relationships) will have similar Node2Vec embeddings.


## Part 6: Computing Similarity Between Nodes

You can compute cosine similarity between node embeddings to quantitatively measure how structurally similar two nodes are.


In [17]:
def cosine_similarity(vec1: list, vec2: list) -> float:
    """Compute cosine similarity between two vectors."""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

# Compare similarity between pairs of nodes
print("Computing structural similarity between node pairs:")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    node_pairs = [
        ("Alice Johnson", "Bob Smith"),      # Both work at TechCorp, know each other
        ("Alice Johnson", "Carol Davis"),    # Both work at TechCorp, know each other
        ("Bob Smith", "Carol Davis"),        # Both work at TechCorp, know each other
        ("TechCorp", "CloudTech"),           # Both organizations
        ("Alice Johnson", "David Chen"),     # Different companies
        ("TechCorp", "DataSystems Inc"),     # Both organizations, but different connections
    ]
    
    for node1_name, node2_name in node_pairs:
        node1 = store.get_node(node1_name)
        node2 = store.get_node(node2_name)
        
        if node1 and node2 and node1.get('vector_index') and node2.get('vector_index'):
            emb1_data = vector_store.get(node1['vector_index'])
            emb2_data = vector_store.get(node2['vector_index'])
            
            if emb1_data and emb2_data:
                similarity = cosine_similarity(emb1_data['embedding'], emb2_data['embedding'])
                print(f"{node1_name} <-> {node2_name}")
                print(f"  Structural similarity: {similarity:.4f} (1.0 = identical, 0.0 = orthogonal)")
                
                # Show why they're similar/different
                node1_name_upper = node1['name'].upper()
                node2_name_upper = node2['name'].upper()
                conn1 = store.query_cypher(f"""
                    MATCH (n:Entity {{name: '{node1_name_upper}'}})-[r:Relationship]-(other:Entity)
                    RETURN COUNT(*) as degree
                """)
                conn2 = store.query_cypher(f"""
                    MATCH (n:Entity {{name: '{node2_name_upper}'}})-[r:Relationship]-(other:Entity)
                    RETURN COUNT(*) as degree
                """)
                
                degree1 = conn1[0]['degree'] if conn1 else 0
                degree2 = conn2[0]['degree'] if conn2 else 0
                print(f"  Node degrees: {node1_name} ({degree1}), {node2_name} ({degree2})")
                print()


Computing structural similarity between node pairs:
----------------------------------------------------------------------
Alice Johnson <-> Bob Smith
  Structural similarity: 0.9988 (1.0 = identical, 0.0 = orthogonal)
  Node degrees: Alice Johnson (3), Bob Smith (4)

Alice Johnson <-> Carol Davis
  Structural similarity: 0.9984 (1.0 = identical, 0.0 = orthogonal)
  Node degrees: Alice Johnson (3), Carol Davis (3)

Bob Smith <-> Carol Davis
  Structural similarity: 0.9992 (1.0 = identical, 0.0 = orthogonal)
  Node degrees: Bob Smith (4), Carol Davis (3)

TechCorp <-> CloudTech
  Structural similarity: 0.0949 (1.0 = identical, 0.0 = orthogonal)
  Node degrees: TechCorp (3), CloudTech (1)

Alice Johnson <-> David Chen
  Structural similarity: 0.1232 (1.0 = identical, 0.0 = orthogonal)
  Node degrees: Alice Johnson (3), David Chen (1)

TechCorp <-> DataSystems Inc
  Structural similarity: 0.9988 (1.0 = identical, 0.0 = orthogonal)
  Node degrees: TechCorp (3), DataSystems Inc (1)



## Part 7: Working with Embeddings Directly

You can retrieve embeddings directly from the VectorStore and work with them for custom applications (clustering, visualization, dimensionality reduction, etc.).


In [18]:
# Get all nodes and their embeddings for analysis
print("Retrieving all node embeddings for analysis:")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    # Get all nodes from the graph
    all_nodes_query = """
    MATCH (e:Entity)
    RETURN e.name, e.type, e.vector_index
    """
    
    nodes_data = store.query_cypher(all_nodes_query)
    
    print(f"Retrieved {len(nodes_data)} nodes with embeddings\n")
    
    # Get embeddings for each node
    embeddings_data = []
    for node_row in nodes_data:
        node_name = node_row['e.name']
        vector_index = node_row.get('e.vector_index')
        
        if vector_index:
            embedding_info = vector_store.get(vector_index)
            if embedding_info:
                embeddings_data.append({
                    'name': node_name,
                    'type': node_row['e.type'],
                    'embedding': embedding_info['embedding'],
                    'embedding_dim': len(embedding_info['embedding'])
                })
    
    print(f"Successfully retrieved {len(embeddings_data)} embeddings\n")
    
    # Display summary statistics
    print("Embedding Statistics:")
    if embeddings_data:
        print(f"  Total nodes with embeddings: {len(embeddings_data)}")
        print(f"  Embedding dimension: {embeddings_data[0]['embedding_dim']}")
        print(f"  Node types: {set(d['type'] for d in embeddings_data)}")
        print()
        
        # Show sample embeddings
        print("Sample embeddings (first 5 nodes):")
        for i, data in enumerate(embeddings_data[:5], 1):
            print(f"{i}. {data['name']} ({data['type']})")
            print(f"   First 5 values: {[round(x, 4) for x in data['embedding'][:5]]}")
            print(f"   Embedding norm: {np.linalg.norm(np.array(data['embedding'])):.4f}")
            print()


Retrieving all node embeddings for analysis:
----------------------------------------------------------------------
Retrieved 7 nodes with embeddings

Successfully retrieved 7 embeddings

Embedding Statistics:
  Total nodes with embeddings: 7
  Embedding dimension: 128
  Node types: {'Person', 'Organization'}

Sample embeddings (first 5 nodes):
1. ALICE JOHNSON (Person)
   First 5 values: [-0.0343, -0.1868, 0.2308, -0.0416, -0.0476]
   Embedding norm: 1.8516

2. BOB SMITH (Person)
   First 5 values: [-0.0386, -0.1998, 0.2313, -0.0276, -0.0547]
   Embedding norm: 1.9718

3. CAROL DAVIS (Person)
   First 5 values: [-0.0379, -0.1992, 0.2112, -0.0421, -0.0425]
   Embedding norm: 1.8841

4. DAVID CHEN (Person)
   First 5 values: [0.1062, -0.4307, 0.1815, -0.4973, -0.02]
   Embedding norm: 2.9783

5. TECHCORP (Organization)
   First 5 values: [-0.039, -0.1924, 0.2222, -0.0287, -0.0474]
   Embedding norm: 1.8909



## Part 8: Updating Node Embeddings After Graph Changes

When you modify the graph structure (add/remove edges, change connections), Node2Vec embeddings need to be recomputed to reflect the new structure. Unlike text-based embeddings, Node2Vec embeddings are based on graph topology, not node attributes.


In [ ]:
# Demonstrate how graph structure changes affect embeddings
print("Demonstrating how graph structure changes affect Node2Vec embeddings:")
print("-" * 70)

with GraphStore(db_path=graph_name) as store:
    # Get current similarity for Alice and Bob
    alice = store.get_node("Alice Johnson")
    bob = store.get_node("Bob Smith")
    
    if alice and bob and alice.get('vector_index') and bob.get('vector_index'):
        emb1 = vector_store.get(alice['vector_index'])
        emb2 = vector_store.get(bob['vector_index'])
        
        if emb1 and emb2:
            similarity_before = cosine_similarity(emb1['embedding'], emb2['embedding'])
            print(f"Before adding edge:")
            print(f"  Alice Johnson <-> Bob Smith similarity: {similarity_before:.4f}")
            print()
            
            # Add a new connection between Alice and Bob
            # (They already know each other, but let's add a "collaborates_with" relationship)
            store.add_edge("Alice Johnson", "collaborates_with", "Bob Smith")
            print(f"✓ Added new edge: Alice Johnson -> collaborates_with -> Bob Smith")
            print()
            
            # Recompute embeddings to reflect the new structure
            print("Recomputing embeddings with updated graph structure...")
            store.compute_graph_embeddings(
                vector_store,
                dimensions=128,
                num_walks=10,
                walk_length=80
            )
            print("✓ Embeddings recomputed")
            print()
            
            # Check similarity again
            alice_updated = store.get_node("Alice Johnson")
            bob_updated = store.get_node("Bob Smith")
            
            if alice_updated.get('vector_index') and bob_updated.get('vector_index'):
                emb1_new = vector_store.get(alice_updated['vector_index'])
                emb2_new = vector_store.get(bob_updated['vector_index'])
                
                if emb1_new and emb2_new:
                    similarity_after = cosine_similarity(emb1_new['embedding'], emb2_new['embedding'])
                    print(f"After adding edge:")
                    print(f"  Alice Johnson <-> Bob Smith similarity: {similarity_after:.4f}")
                    print(f"  Change: {similarity_after - similarity_before:+.4f}")
                    print()
                    print("Note: Adding an edge typically increases similarity between nodes,")
                    print("as they now share more structural patterns.")


## Part 9: Summary

This notebook demonstrated Node2Vec-based graph structure-aware embeddings and how to use them for similarity search.


In [20]:
# Cleanup (uncomment to run)
vector_store.close()

with GraphStore(db_path=graph_name) as store:
    store.delete_graph()
    print("Graph deleted")


Graph deleted
